In [3]:
import pandas as pd
df = pd.read_csv("survey.csv")

In [4]:
df

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


Initial thoughts:
The comments seems to be mostly NaN values, so I'll get rid of that column.
I don't really care about the time, so I'll drop that too. 
Gender seems to be in a lot of different formats. I wonder what the unique values of each are? 

In [5]:
df = df.drop(columns=["Timestamp","comments"])

Now let's deal with the other columns.

In [31]:
for c in df.columns:
    print("unique values of", c, "are: \n", df[c].unique())

unique values of Age are: 
 [         37          44          32          31          33          35
          39          42          23          29          36          27
          46          41          34          30          40          38
          50          24          18          28          26          22
          19          25          45          21         -29          43
          56          60          54         329          55 99999999999
          48          20          57          58          47          62
          51          65          49       -1726           5          53
          61           8          11          -1          72]
unique values of Gender are: 
 ['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'All' 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-f

Thoughts: I could convert a lot of these to booleans, if I felt like it, but I probably shouldn't because of the "I don't know" and "Maybe" answers.
There's a lot of categorical data, in fact, it's mostly categorical data.
The gender seemed to be a "fill in your own" one. I'm wondering if I could get counts on each?
In terms of age, I probably want to get rid of the negative values, unless there's a lot of them, plus that 99999 one. 

I wonder how many total entries I have? I'll figure it out below.

In [32]:
df.size

31475

**that's wrong**, I'll figure out later because I just scrolled back up and it's 1259 rows which is not a whole lot, but it's easy to work with. Now let's deal with the gender thing. Googled a bit and found value_counts() so we'll see if it works.

In [33]:
df.Gender.value_counts()

Male                                              615
male                                              206
Female                                            121
M                                                 116
female                                             62
F                                                  38
m                                                  34
f                                                  15
Make                                                4
Male                                                3
Woman                                               3
Female                                              2
Female (trans)                                      2
Man                                                 2
Cis Male                                            2
cis male                                            1
woman                                               1
msle                                                1
Female (cis)                

perfect, that does exactly what I want it to do.
I think I'll make a "gender non-conforming" category for anyone that didn't answer strongly in the male or female direction. "Male-ish", and "Female (trans)" will get classified as male and female, respectively, whereas "genderqueer" and "male-leaning androgynous" will be in "gender non-conforming". I'll just drop stuff I can't figure out, like "a little about you" and "p". I wonder what would happen if I excluded gender from the analysis. We'll see later.

Looking at the data and data description again, work_infere looks interesting but I'll have to look closer at it. I should probably figure out what I want my question to be before I look any farther. Then I can do some EDA.

In [6]:
df.work_interfere.describe()

count           995
unique            4
top       Sometimes
freq            465
Name: work_interfere, dtype: object